## **Domain**

This project operates within the domain of Sign Language Recognition for Emergency Response Systems, an intersection of computer vision, gesture recognition, and assistive technology. The system leverages real-time hand gesture detection and classification to enable effective communication in high-stakes situations.

# PROJECT : SIGN LANGUAGE RECOGNITION

## **Problem Statement**

The primary objective of this dataset is to enable the development of a sign language recognition system that assists the deaf and hard-of-hearing community in emergency scenarios. The system should:
- Recognize specific hand gestures captured via a smartphone camera.
- Map these gestures to predefined emergency classes such as "Danger," "Help," and "Police."
- Facilitate non-verbal communication by triggering automated alerts to relevant authorities or services.

## **Real-Life Use Case**

### Mobile Emergency Application
An application that integrates real-time sign language recognition, allowing individuals to communicate during emergencies. The app will:
- Recognize specific gestures related to emergency situations.
- Trigger notifications to relevant emergency services.

**Example Scenario**
A deaf person trapped in a building during a fire performs the "Fire" gesture in front of their smartphone camera. The app detects the gesture and sends an alert to the fire department, including the person’s location.

---

## **Dataset Description**

The dataset is designed to support the development of a mobile emergency application for sign language recognition. It contains images and videos representing 8 distinct emergency-related gestures, ensuring inclusivity and practicality for individuals who are deaf or hard of hearing. The dataset emphasizes diverse environmental conditions to enhance model robustness.

## **Classes**

The dataset includes the following classes:

- **Danger** : Indicates a potential threat or hazard.
- **Help**: Requests immediate assistance.
- **Stop**: Signals a halt or warning.
- **Please**: Indicates the urge to ask someone to help them do something.
- **Fire**: Communicates the presence of a fire.
- **Water**: Represents the need for water in emergency situations.
- **Police**: Indicates the need for law enforcement.
- **Emergency**: Represents a general state of urgency.

## **Data Collection**
- Collect video frames of 8 emergency gestures performed by diverse individuals.
- Ensure varied hand orientations, lighting conditions, and backgrounds to improve robustness.
- Include diverse demographics to account for variations in hand size and shape.

---

## **Dataset Preparation Steps**

- *Data Unzipping*
    - Extract raw data from compressed files.
- *Frame Extraction*
    - Convert video data into individual frames for analysis.
- *Upsampling*
    - Balance the dataset by increasing the number of underrepresented class samples.
- *Preprocessing Frames* with Hand Detection and Cropping (ROI Extraction)
    - Extract Region of Interest (**ROI**) focusing on hand regions.
    - **Resize** images to a consistent dimension (e.g., 224x224 pixels).
    - **Normalize** pixel values to enhance model performance.
    - Convert images to **grayscale** for simplicity.
    - Apply **Canny edge detection** for emphasizing hand contours.

- *Balanced Augmentation*
    - Apply rotation, flipping, scaling, and synthetic noise to simulate diverse conditions and improve robustness.


---

## PART 1 : PRE-PROCESSING

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Unzipping the Dataset**

In [ ]:
import zipfile #  Provides tools for working with `.zip` files.
import os # Used for interacting with the file system

# Specify the path to the zip file containing the dataset on Google Drive
zip_path = '/content/drive/MyDrive/Sign Language Recognition/Dataset.zip'

# Specify the directory where the extracted files will be saved
extract_path = '/content/drive/MyDrive/Sign Language Recognition/'

# Open the zip file in read mode using the zipfile module
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents of the zip file to the specified directory
    zip_ref.extractall(extract_path)

# Print a confirmation message after successful extraction
print("Dataset unzipped!")

Dataset unzipped!


### **Extracting Frames from Videos**

This script extracts frames from videos stored in a structured folder, organized by classes, at a frame rate of 30 FPS. The extracted frames are saved in a corresponding structured output folder.

In [ ]:
import cv2
import glob

def extract_frames_from_videos(input_folder, output_folder, fps=30):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all class folders in the input folder
    for class_folder in os.listdir(input_folder):
        class_path = os.path.join(input_folder, class_folder)  # Path to the class folder
        output_class_folder = os.path.join(output_folder, class_folder)  # Corresponding output folder
        os.makedirs(output_class_folder, exist_ok=True)  # Create output folder for the class

        # Process all video files in the current class folder
        for video_file in glob.glob(f"{class_path}/*.mp4"):
            cap = cv2.VideoCapture(video_file)  # Open the video file
            video_name = os.path.splitext(os.path.basename(video_file))[0]  # Get video name without extension
            frame_rate = int(cap.get(cv2.CAP_PROP_FPS))  # Get the video frame rate
            frame_interval = max(1, frame_rate // fps)  # Calculate interval for extracting frames
            frame_count = 0  # Count of frames processed
            frame_index = 0  # Index for extracted frames

            # Read frames from the video
            while cap.isOpened():
                ret, frame = cap.read()  # Read the next frame
                if not ret:  # Break the loop if no more frames
                    break
                if frame_count % frame_interval == 0:  # Check if the frame should be saved
                    frame_name = f"{output_class_folder}/{video_name}_frame_{frame_index}.jpg"  # Frame file name
                    cv2.imwrite(frame_name, frame)  # Save the frame as an image
                    frame_index += 1  # Increment frame index
                frame_count += 1  # Increment frame count
            cap.release()  # Release the video capture object
    print("Frames extracted!")  # Confirmation message

# Paths for input videos and output frames
input_videos_path = '/content/drive/MyDrive/Sign Language Recognition/Dataset'
output_frames_path = '/content/drive/MyDrive/Sign Language Recognition/Frame_extracted_data'

# Extract frames at 30 FPS
extract_frames_from_videos(input_videos_path, output_frames_path)

Frames extracted!


### **Counting Frames per Class**

This script calculates the number of extracted frames for each class in the specified folder. The results are returned as a dictionary, where the keys are class names and the values are the corresponding frame counts.

In [ ]:
def count_frames_per_class(folder_path):
    frame_counts = {}  # Dictionary to store the count of frames for each class

    # Iterate through each class folder in the specified folder path
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)  # Path to the current class folder

        # Count the number of .jpg files (frames) in the class folder
        frame_counts[class_folder] = len(glob.glob(f"{class_path}/*.jpg"))

    # Return the dictionary containing frame counts for each class
    return frame_counts

# Call the function to count frames in the extracted frames folder
frame_counts = count_frames_per_class(output_frames_path)

# Print the frame counts for each class
print("Frame counts per class:", frame_counts)

Frame counts per class: {'danger': 432, 'emergency': 273, 'fire': 180, 'help': 604, 'please': 523, 'police': 496, 'stop': 304, 'water': 676}


---

###  **Upsampling Classes with Augmentation**

This script balances the dataset by upsampling classes with fewer frames. It uses specified augmentations to generate new frames and ensures all classes have the same number of frames.

In [ ]:
import random

def augment_frame(frame):
    # Apply one of the specified augmentations to the frame
    augmentations = [
        lambda x: cv2.flip(x, 1),  # Horizontal flip
        lambda x: cv2.rotate(x, cv2.ROTATE_90_CLOCKWISE),  # Rotate 90 degrees clockwise
        lambda x: cv2.rotate(x, cv2.ROTATE_90_COUNTERCLOCKWISE),  # Rotate 90 degrees counterclockwise
    ]
    # Randomly select and apply an augmentation
    augmentation = random.choice(augmentations)
    return augmentation(frame)

def upsample_classes_with_augmentation(folder_path, frame_counts):
    # Determine the maximum number of frames among all classes
    max_frames = max(frame_counts.values())

    # Iterate through each class and its frame count
    for class_folder, count in frame_counts.items():
        class_path = os.path.join(folder_path, class_folder)  # Path to the class folder
        if count < max_frames:
            # Get a list of all existing frames in the class folder
            current_files = glob.glob(f"{class_path}/*.jpg")
            for i in range(max_frames - count):  # Calculate how many frames need to be added
                # Choose a random frame from the existing files
                source_file = random.choice(current_files)
                frame = cv2.imread(source_file)  # Read the selected frame

                # Apply augmentation to create a new frame
                augmented_frame = augment_frame(frame)

                # Save the augmented frame with a new name
                target_file = f"{class_path}/aug_{i}.jpg"
                cv2.imwrite(target_file, augmented_frame)
    print("Upsampling with specified augmentations complete!")

# Perform upsampling using the function
upsample_classes_with_augmentation(output_frames_path, frame_counts)

# Recount the frames in each class after upsampling
updated_frame_counts = count_frames_per_class(output_frames_path)

# Print the updated frame counts
print("Updated frame counts per class:", updated_frame_counts)

Upsampling with specified augmentations complete!
Updated frame counts per class: {'danger': 676, 'emergency': 676, 'fire': 676, 'help': 676, 'please': 676, 'police': 676, 'stop': 676, 'water': 676}


---